In [1]:
from diffusers import StableDiffusionPipeline
import torch
import os

# os.environ["CUDA_VISIBLE_DEVICES"]="0"
num_gpus = torch.cuda.device_count()
print(f"Number of visible GPUs: {num_gpus}")

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]


Number of visible GPUs: 1


In [28]:
#---- Load SD-v1.5 Model ----
model_id = "sd-legacy/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")

Loading pipeline components...: 100%|██████████| 7/7 [00:01<00:00,  5.54it/s]


In [29]:
prompt = "photo of a chest x-ray"
image = pipe(prompt).images[0]  
image.save("figures/cxr_before_ft.png")

# prompt = "image of a female chest x-ray"
# image = pipe(prompt).images[0]  
# image.save("female_cxr.png")

100%|██████████| 50/50 [00:01<00:00, 29.39it/s]


## Fine-Tuning SD-v1.5 with DreamBooth

In [11]:
from accelerate import Accelerator
from diffusers import DiffusionPipeline

# Load the pipeline with the same arguments (model, revision) that were used for training
model_id = "CompVis/stable-diffusion-v1-4"
pipeline = DiffusionPipeline.from_pretrained(model_id)

# accelerator = Accelerator()

# # Use text_encoder if `--train_text_encoder` was used for the initial training
# unet, text_encoder = accelerator.prepare(pipeline.unet, pipeline.text_encoder)

# # Restore state from a checkpoint path. You have to use the absolute path here.
# accelerator.load_state("/sddata/dreambooth/daruma-v2-1/checkpoint-100")

# # Rebuild the pipeline with the unwrapped models (assignment to .unet and .text_encoder should work too)
# pipeline = DiffusionPipeline.from_pretrained(
#     model_id,
#     unet=accelerator.unwrap_model(unet),
#     text_encoder=accelerator.unwrap_model(text_encoder),
# )

# # Perform inference, or save, or push to the hub
# pipeline.save_pretrained("dreambooth-pipeline")

/home/runai-home/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading pipeline components...: 100%|██████████| 7/7 [00:00<00:00,  9.62it/s]


In [23]:
from diffusers.utils.torch_utils import randn_tensor
from diffusers.training_utils import compute_snr

ImportError: cannot import name 'compute_snr' from 'diffusers.training_utils' (/home/runai-home/.local/lib/python3.10/site-packages/diffusers/training_utils.py)

In [31]:
from diffusers import DiffusionPipeline
import torch

pipeline = DiffusionPipeline.from_pretrained("saved_models", torch_dtype=torch.float16, use_safetensors=True).to("cuda")
image = pipeline("photo of a chest x-ray", num_inference_steps=50, guidance_scale=7.5).images[0]
image.save("figures/cxr_after_40000ft.png")

100%|██████████| 50/50 [00:01<00:00, 30.00it/s]
